### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [2]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [10]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('text_data/dataset.csv')

In [5]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [35]:
X = df['text']
y = df['humor'].astype(int)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=24)

#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [36]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [38]:
def lemmatize(text):
  return ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])

def stem(text):
  return ' '.join([stemmer.stem(w) for w in word_tokenize(text)])

In [39]:
Xlemmatized = X.apply(lemmatize)
Xstemmed = X.apply(stem)

In [48]:
# Lemmatized data sets
X_trainLem, X_testLem, y_train, y_test = train_test_split(Xlemmatized, y, test_size=0.25, random_state=24)

# Stemmed data sets
X_trainStem, X_testStem, y_train, y_test = train_test_split(Xstemmed, y, test_size=0.25, random_state=24)

#### Model Training

In [49]:
# Logistic
logisticParams = {
  'pp__max_features': [500, 1000, 2000, 5000],
  'pp__stop_words': ['english', None]
}

logisticPipeBow = Pipeline([
  ('pp', CountVectorizer()),
  ('logistic', LogisticRegression())
])

logisticPipeTfIdf = Pipeline([
  ('pp', TfidfVectorizer()),
  ('logistic', LogisticRegression())
])

# Bow
logisticGridLemmatizedBow = GridSearchCV(logisticPipeBow, param_grid=logisticParams)
logisticGridStemmedBow = GridSearchCV(logisticPipeBow, param_grid=logisticParams)

logisticGridLemmatizedBow.fit(X_trainLem, y_train)
logisticGridStemmedBow.fit(X_trainStem, y_train)

# TfIdf
logisticGridLemmatizedTfIdf = GridSearchCV(logisticPipeTfIdf, param_grid=logisticParams)
logisticGridStemmedTfIdf = GridSearchCV(logisticPipeTfIdf, param_grid=logisticParams)

logisticGridLemmatizedTfIdf.fit(X_trainLem, y_train)
logisticGridStemmedTfIdf.fit(X_trainStem, y_train)

GridSearchCV(estimator=Pipeline(steps=[('pp', TfidfVectorizer()),
                                       ('logistic', LogisticRegression())]),
             param_grid={'pp__max_features': [500, 1000, 2000, 5000],
                         'pp__stop_words': ['english', None]})

In [ ]:
## Code below is untested

In [ ]:
# DT
dtParams = {
  'pp__max_features': [500, 1000, 2000, 5000],
  'pp__stop_words': ['english', None],
  'dt__max_depth': [1000, 5000]
}

dtPipeBow = Pipeline([
  ('pp', CountVectorizer()),
  ('dt', DecisionTreeClassifier())
])

dtPipeTfIdf = Pipeline([
  ('pp', TfidfVectorizer()),
  ('dt', DecisionTreeClassifier())
])

# Bow
dtGridLemmatizedBow = GridSearchCV(dtPipeBow, param_grid=logisticParams)
dtGridStemmedBow = GridSearchCV(dtPipeBow, param_grid=logisticParams)

dtGridLemmatizedBow.fit(X_trainLem, y_train)
dtGridStemmedBow.fit(X_trainStem, y_train)

# TfIdf
dtGridLemmatizedTfIdf = GridSearchCV(dtPipeTfIdf, param_grid=logisticParams)
dtGridStemmedTfIdf = GridSearchCV(dtPipeTfIdf, param_grid=logisticParams)

dtGridLemmatizedTfIdf.fit(X_trainLem, y_train)
dtGridStemmedTfIdf.fit(X_trainStem, y_train)

In [ ]:
# Naive Bayes
nbParams = {
  'pp__max_features': [500, 1000, 2000, 5000],
  'pp__stop_words': ['english', None],
  'nb__alpha': [1.0, 5.0, 10.0]
}

nbPipeBow = Pipeline([
  ('pp', CountVectorizer()),
  ('nb', MultinomialNB())
])

nbPipeTfIdf = Pipeline([
  ('pp', TfidfVectorizer()),
  ('nb', MultinomialNB())
])

# Bow
nbGridLemmatizedBow = GridSearchCV(logisticPipeBow, param_grid=logisticParams)
nbGridStemmedBow = GridSearchCV(logisticPipeBow, param_grid=logisticParams)

nbGridLemmatizedBow.fit(X_trainLem, y_train)
nbGridStemmedBow.fit(X_trainStem, y_train)

# TfIdf
nbGridLemmatizedTfIdf = GridSearchCV(logisticPipeTfIdf, param_grid=logisticParams)
nbGridStemmedTfIdf = GridSearchCV(logisticPipeTfIdf, param_grid=logisticParams)

nbGridLemmatizedTfIdf.fit(X_trainLem, y_train)
nbGridStemmedTfIdf.fit(X_trainStem, y_train)

In [51]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'],
             'best_params': ["{'max_features': 3000, 'stop_words': None}", "{'max_features': 2000, 'stop_words': None}", "{'max_features': 3000, 'stop_words': None}"],
             'best_score': ['0.942', '0.927', '0.898'],
             'best_version': ['Bag of Words', 'Bag of Words', 'Bag of Words'],
              'best time': ['4 minutes','21 minutes','3 minutes']
              }).set_index('model')

,best_params,best_score,best_version,best time
model,,,,
Logistic,"{'max_features': 3000, 'stop_words': None}",0.942,Bag of Words,4 minutes
Decision Tree,"{'max_features': 2000, 'stop_words': None}",0.927,Bag of Words,21 minutes
Bayes,"{'max_features': 3000, 'stop_words': None}",0.898,Bag of Words,3 minutes
